**Sentiment Integration:** <br>This is the module to shape the sentiment data for addition to the primary dataframe. The output of this script is three new csv files: one where the data is shaped to the financials collected (XXX_financials.csv), one is indexed based on the timepoints that contain sentiment data (XXX_sentiment.csv), as we only have collected sentiment data once per day, and only a year's worth. The last is the sentiment table joined to technical data, indexed based on the smaller technical dataset.

In [1]:
import pandas as pd
import datetime

In [37]:
#loads pre-collected CSVs,cleans. Sent is the sentiment data, data is financials.
sent=pd.read_csv('raw_data/XLV.csv')
sent.index=pd.to_datetime(sent['asof_date'].astype(str) + ' 09:30')
sent.index.name=None
sent_clean=sent[['total_scanned_messages','bull_bear_msg_ratio','bull_minus_bear']]
data=pd.read_csv('raw_data/20161114_XLV.csv')
data.index=pd.to_datetime(data['Unnamed: 0'])
data.index.name=None
del data['Unnamed: 0']

In [38]:
#makes a new df, sentiment, that indexes based on sentiment data (1x/day, to 09/2015)
sentiment=data.join(sent_clean, how = 'right', lsuffix='_x')


#makes a new df, finance, that indexes based on financial data (2x/day, to 01/2009)
finance=data.join(sent_clean, how = 'left', lsuffix='_x')


In [9]:
#writes dataframes to new file with corresponding label
sentiment.to_csv("ANIK_sentiment.csv")
finance.to_csv("ANIK_financials.csv")

In [28]:
sentiment.describe()

,Price,7,50,200,750g,50200g,7200g,750d,50200d,7200d,total_scanned_messages,bull_bear_msg_ratio,bull_minus_bear
count,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,420.000000,420.000000,420.000000
mean,70.271883,70.303873,70.410466,70.800520,32.166096,574.469178,110.414384,34.998288,135.917808,79.325342,11.130952,1.047492,0.510566
std,2.674697,2.578617,2.193163,1.598350,22.706872,475.583999,93.782419,24.441792,83.125821,54.632515,7.366602,1.698779,1.288268
min,62.750000,63.810001,66.176324,68.766323,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-3.000000
25%,68.247500,68.108500,69.096389,69.111219,13.500000,73.250000,35.875000,13.875000,63.250000,33.375000,6.000000,0.000000,-0.198750
50%,70.575001,70.583501,70.585338,70.838281,28.750000,948.500000,76.000000,32.000000,135.000000,69.750000,9.000000,0.000000,0.130000
75%,72.125002,71.989750,71.764642,72.097053,46.625000,1020.250000,148.750000,53.000000,207.750000,121.250000,15.000000,1.500000,1.677500
max,75.739998,75.349999,74.523662,73.872101,92.000000,1093.000000,297.500000,90.000000,280.500000,194.000000,40.000000,11.000000,3.330000


In [39]:
#loads technical data, adds complementary indices, and trims unnecessary data
tech=pd.read_csv('raw_data/XLV_technical_data.csv')
tech.index=pd.to_datetime(tech['Period'].str.replace('00:00:00', '09:30:00'))
techj=tech[['Aroon Oscillator [25]','CMF [20]','MACD (12,26,9) - DIVERGENCE','MACD (12,26,9) - MACD','MACD (12,26,9) - SIGNAL','Williams %R [14]']]

In [40]:
#joins sentiment and technical data, saves to csv
techsent=sentiment.join(techj, how = 'left', lsuffix='_x')
techsent.index.name='date'
techsent.to_csv("org_data/XLV_technicals.csv")